In [1]:
#fast chekc to see if IP matches 
import requests
print(requests.get("https://api.ipify.org").text)

37.67.198.11


If the video has an available transcript, retrieves the full transcript text using youtube_transcript_api,
otherwise, returns the raised exception if no transcript available

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs
import logging

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s",
    filename="yt_api.log",
    filemode="a",  # append
)

logger = logging.getLogger(__name__)

In [4]:
# video_url = "https://www.youtube.com/watch?v=GuM6dQGRFyQ"
# video_url = "https://www.youtube.com/watch?v=-hkEzwaY754"
# video_url = "https://www.youtube.com/watch?v=HhzpDM36Pzg"
video_url = "https://www.youtube.com/watch?v=01rldii7b_Q"

In [5]:
def get_youtube_video_id(url: str) -> str | None:
    parsed = urlparse(url)

    # youtu.be/<id>
    if parsed.netloc in ("youtu.be", "www.youtu.be"):
        return parsed.path.lstrip("/")

    # youtube.com/*
    if "youtube.com" in parsed.netloc:
        # watch?v=<id>
        if parsed.path == "/watch":
            return parse_qs(parsed.query).get("v", [None])[0]

        # /embed/<id>, /shorts/<id>
        parts = parsed.path.split("/")
        if len(parts) >= 3 and parts[1] in {"embed", "shorts"}:
            return parts[2]

    return None

def retrieve_video_transcript(video_url:str) -> str:
    video_id = get_youtube_video_id(video_url)
    ytt_api = YouTubeTranscriptApi()
    full_text=""
    try : 
        fetched_transcript = ytt_api.fetch(video_id)
    except Exception as e:
        logger.exception(
            "Failed to fetch transcript for video_url=%s",
            video_url,
        )
        return("")
    for snippet in fetched_transcript.snippets:
        full_text+= snippet.text + " "
    return(full_text)

In [6]:
test_text = retrieve_video_transcript(video_url)

In [7]:
test_text

"Today we're going to talk about the latest innovations made available through the C3 Agentic AI platform to help developers onboard and build, deploy, and operate applications more quickly. Um, and as Jim Snobby talked about this morning, that's really the entire development team. Um, and so I'm joined today by my colleague Josh Prisbilo, who is the lead product manager on the C3 Agentic AI platform. Uh, and we're going to live demo some software to you today. My name is Jake Whitam. I lead developer experience at C3. Um, and thank you for joining us. So, one of the things that gets me inspired every year are the stories and the use cases that our customers are presenting on stage. And you know these are across supply network risk, inventory optimization, predictive maintenance, demand forecasting and more. And it's the C3 AI applications that are pre-built and extended in most cases to achieve these use cases or built you know from scratch on top of the C3 Aentic AI platform and we'r

In [11]:
with open(f"{get_youtube_video_id(video_url)}.txt", "w", encoding="utf-8") as f:
    f.write(test_text)